# SRS and FRep 接口

## 引入工具

In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

def _IMPORT_(x):
    try:
        exec(x, globals())
    except:
        pass

###
### Common ###
###

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()
    

###
### Display ###
###

_IMPORT_('import cv2')
_IMPORT_('from PIL import Image')
_IMPORT_('from torchvision.utils import make_grid')
_IMPORT_('import matplotlib.pyplot as plt')
_IMPORT_('import plotly')
_IMPORT_('import plotly.graph_objects as go')
_IMPORT_('import ipywidgets as widgets')
_IMPORT_('from ipywidgets import interact, interactive, fixed, interact_manual')

# plotly.offline.init_notebook_mode(connected=False)

plt.rcParams['figure.figsize'] = (12.0, 8.0)

def show_table(headers, data, width=900):
    ncols = len(headers)
    width = int(width / ncols)
    lralign = []
    caption = []
    for item in headers:
        astr = ''
        if item[0] == ':':
            astr = ':'
            item = item[1:]
        astr += '---'
        if item[-1] == ':':
            astr += ':'
            item = item[:-1]
        lralign.append(astr)
        caption.append(item)
    captionstr = '|'.join(caption) + chr(10)
    lralignstr = '|'.join(lralign) + chr(10)
    imgholdstr = '|'.join(['<img width=%d/>' % width] * ncols) + chr(10)
    table = captionstr + lralignstr + imgholdstr
    is_dict = isinstance(data[0], dict)
    for row in data:
        if is_dict:
            table += '|'.join([f'{row[c]}' for c in caption]) + chr(10)
        else:
            table += '|'.join([f'{col}' for col in row]) + chr(10)
    return Markdown(table)

def show_video(vidsrc, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if vidsrc.startswith('http'):
        data_url = vidsrc
    else:
        mp4 = open(vidsrc, 'rb').read()
        data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<video %s %s controls src="%s" type="video/mp4"/>' % (W, H, data_url))

def show_image(imgsrc, width=None, height=None):
    if isinstance(imgsrc, np.ndarray):
        img = imgsrc
        if width or height:
            if width and height:
                size = (width, height)
            else:
                rate = img.shape[1] / img.shape[0]
                if width:
                    size = (width, int(width/rate))
                else:
                    size = (int(height*rate), height)
            img = cv2.resize(img, size)
            plt.figure(figsize=(3*int(size[0]/80+1), 3*int(size[1]/80+1)), dpi=80)
        plt.axis('off')
        if len(img.shape) > 2:
            plt.imshow(img);
        else:
            plt.imshow(img, cmap='gray');
        return

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if imgsrc.startswith('http'):
        data_url = imgsrc
    else:
        if len(imgsrc) > 2048:
            data_url = 'data:image/jpg;base64,' + imgsrc
        else:
            img = open(imgsrc, 'rb').read()
            data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if image_path.startswith('http'):
        data_url = image_path
    else:
        img = open(image_path, 'rb').read()
        data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

def im_read(url, rgb=True, size=None):
    response = requests.get(url)
    if response:
        imgmat = np.frombuffer(response.content, dtype=np.uint8)
        img = cv2.imdecode(imgmat, cv2.IMREAD_COLOR)
        if rgb:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if size:
           if isinstance(size, int):
               size = (size, size)
           img = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
        return img
    return None


CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed


## 地址

### 设备信息

In [2]:
devices = (
    ["00d402530642", "摄像头一", "SSAC-467980-CBBBC", "88555220"],
    ["00e685ef87b4", "摄像头二", "SSAC-462825-DBFBF", "93057463"],
    ["00ee44217446", "摄像头三", "SSSS-120078-DBAFF", "49430446"],
    ["008e9b61ac6c", "摄像头四", "SSAA-147882-DFFFD", "42680711"],
    ["00d51a6becbb", "摄像头五", "SSAC-058581-BCEFF", "35354379"]
)

show_table(['MAC', 'Label', 'UUID', 'CODE'], devices)

MAC|Label|UUID|CODE
---|---|---|---
<img width=225/>|<img width=225/>|<img width=225/>|<img width=225/>
00d402530642|摄像头一|SSAC-467980-CBBBC|88555220
00e685ef87b4|摄像头二|SSAC-462825-DBFBF|93057463
00ee44217446|摄像头三|SSSS-120078-DBAFF|49430446
008e9b61ac6c|摄像头四|SSAA-147882-DFFFD|42680711
00d51a6becbb|摄像头五|SSAC-058581-BCEFF|35354379


### 直播推流地址

```
URL: `rtmp://116.85.58.226:1935/live/00d402530642?vhost=seg.300s`
               SRS地址    端口  任意    MAC地址       参数
                                |                    |
                           (推荐用工厂ID)     (配置保存mp4视频的时长)
```

### 直播拉流地址(HLS)

```
URL: `http://116.85.58.226:8080/live/00d402530642.m3u8`
              SRS地址     端口  app    MAC地址
```

### VLC

```
base64('admin:admin') -> `YWRtaW46YWRtaW4=` 

rtmp://172.16.0.103:1935/flash/11:YWRtaW46YWRtaW4= 
            
rtsp://172.16.0.103/11
```

## 接口

### 视频上传OSS通知信息

redis key: `factory:on_dvr`

```json
{
    "etag": "61590abd3fee5b6a346ca61dba2202b8",   // S3对象唯一码
    "bucket": "frepai", 
    "object": "live/00e685ef87b4/20210629/videos/20210629171503.mp4", // 视频存放的Key(位置)
    "size": 306964,         // 视频的大小(字节)
    "time": [
        1624958135.771467,  // 上传起始时间
        1624958135.8189926  // 上传结束时间
    ],  
    "app": "live",             // 直播视频app标识(可以是任意其他字符, 推荐使用工厂号)
    "stream": "00e685ef87b4",  // 直播视频livestream标识, 必须为摄像头的mac地址
    "filename": "20210629171503.mp4",  // 视频的文件名
    "duration": 30                     // 视频的时长(单位s)
}
```

## 远程控制RTMP

### API

#### 远程清空设备推流地址

    curl -X GET "http://ftposs.krzhibo.com/oss/reset?mac=00d402530642" -H "accept: */*"
    
    
#### 查询设备推流地址

    curl -X GET "http://ftposs.krzhibo.com/oss/getRtmpAddr?mac=00d402530642" -H "accept: */*"
    
    
#### 根据mac地址获取推流状态

    curl -X GET "http://ftposs.krzhibo.com/oss/getRtmpStatu?mac=00d402530642" -H "accept: */*"
    
    
#### 获取mac地址
   
    curl -X GET "http://ftposs.krzhibo.com/oss/mac?uid=SSAC-462825-DBFBF" -H "accept: */*"
   

#### 远程修改设备推流地址

    curl -X GET "http://ftposs.krzhibo.com/oss/push?mac=00e685ef87b4&pushurl=rtmp%3A%2F%2F116.85.58.226%3A1935%2Flive%2F00e685ef87b4%3Fvhost%3Dseg.600s" -H "accept: */*"

### Test

In [12]:
SERVER_URL = 'http://ftposs.krzhibo.com'
SERVER_PAGE = f'{SERVER_URL}/swagger-ui.html'
SERVER_RMTP_STATUS = f'{SERVER_URL}/oss/getRtmpStatu?mac=%s'

while True:
    for i in range(2):
        dev = devices[2]
        res = json.loads(requests.get(SERVER_RMTP_STATUS % dev[0]).text)
        if res['code'] != 200:
            raise res
        if 'Publishing' not in res['data']:
            print(dev[0], res)
        time.sleep(1)

00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:47:07", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:47:07", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:47:07", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:47:07", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:47:07", "set": "rtmp://

00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:47:07", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:47:07", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:47:07", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:47:07", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:47:07", "set": "rtmp://

00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:50:30", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:50:30", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:50:30", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:50:30", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:50:30", "set": "rtmp://

00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:50:30", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:50:30", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:50:30", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:50:30", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:50:30", "set": "rtmp://

00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] Initialize IPCAM ... ", "status_time": "21/7/6 14:52:13", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] Initialize IPCAM ... ", "status_time": "21/7/6 14:52:13", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] Initialize IPCAM ... ", "status_time": "21/7/6 14:52:13", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] Initialize IPCAM ... ", "status_time": "21/7/6 14:52:13", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] Initialize IPCAM ... ", "status_time": "21/7/6 14:52:13", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "sta

...

00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:59:09", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:59:09", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:59:09", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:59:09", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:59:09", "set": "rtmp://

00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:59:09", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:59:09", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:59:09", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:59:09", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 14:59:09", "set": "rtmp://

00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 15:01:27", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 15:01:27", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 15:01:27", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 15:01:27", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 15:01:27", "set": "rtmp://

00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 15:01:27", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 15:01:27", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 15:01:27", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 15:01:27", "set": "rtmp://"}', 'msg': '操作成功'}
00ee44217446 {'code': 200, 'data': ' {"result": "2", "type": "X", "status": "V3.2.1 [X][300] on Connecting ..(12) sendtc3a.douyu.com", "status_time": "21/7/6 15:01:27", "set": "rtmp://

KeyboardInterrupt: 

## 本地推流

### 文件

```sh
for((;;)); do \
    ffmpeg -re -i ./test_flower.mp4 \
    -vcodec copy -acodec copy \
    -f flv -y rtmp://116.85.58.226/live/livestream?vhost=seg.30s; \
    sleep 1; \
done
```

### 设备

```sh
sudo ffmpeg -re -i /dev/video0 \
    -s 368x208 -r 15 \
    -acodec aac -ar 32000 -vcodec libx264 \
    -preset:v ultrafast -tune:v zerolatency \
    -f flv rtmp://116.85.58.226/live/livestream
```

### VLC

    base64('admin:admin') -> `YWRtaW46YWRtaW4=` 

    rtmp://172.16.0.103:1935/flash/11:YWRtaW46YWRtaW4= 

    rtsp://172.16.0.103/11